In [ ]:
# 典型策略模式

from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order:  # 上下文

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


# 从ABC派生是为了使用装饰器@abstractmethod
class Promotion(ABC):  # 策略：抽象基类

    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # 第一个具体策略
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):  # 第二个具体策略
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):  # 第三个具体策略
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

# 两个不同积分的顾客，相同的购物车
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

print(Order(joe, cart, FidelityPromo()))
print(Order(ann, cart, FidelityPromo()))
print('---------------')
# 同一个顾客，不同的购物车和折扣策略
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
long_order = [LineItem(str(item_code), 1, 1.0)
               for item_code in range(10)]
print(Order(joe, banana_cart, BulkItemPromo()))
print(Order(joe, long_order, LargeOrderPromo()))
print(Order(joe, cart, LargeOrderPromo()))


In [ ]:
# 上面的各个策略只有一个discount方法没有状态（没有实例属性），可以使用函数来实现

# 不调用discount方法，直接调用策略函数
class Order:  # 上下文

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

print(Order(joe, cart, fidelity_promo))
print(Order(ann, cart, fidelity_promo))
print('---------------')
# 同一个顾客，不同的购物车和折扣策略
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
long_order = [LineItem(str(item_code), 1, 1.0)
               for item_code in range(10)]
print(Order(joe, banana_cart, bulk_item_promo))
print(Order(joe, long_order, large_order_promo))
print(Order(joe, cart, large_order_promo))



In [ ]:
# 计算最佳策略

# 硬编码所有的折扣计算函数
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
print(promos)

# 通过内省模块获得所有折扣计算函数
promos = [globals()[name] for name in globals() 
            if name.endswith('_promo') 
            and name != 'best_promo'] 
print('-----------')
print(promos)

# 将策略函数放到单独的promotions模块中，利用inspect模块获得该模块中所有的折扣方法
#promos = [func for name, func in
#          inspect.getmembers(promotions, inspect.isfunction)]


def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)  # <3>
print(Order(joe, cart, best_promo))
print(Order(ann, cart, best_promo))
print(Order(joe, banana_cart, best_promo))
print(Order(joe, long_order, best_promo))